# Set up notebook
load libraries, etc

In [1]:
# fundamentals
import os, glob, sys
import numpy as np
import pandas as pd
from calendar import monthrange, month_name
import scipy.stats as stats
# import funcs as funcs
import datetime
import imp

# plotting libraries and setup
from matplotlib.colors import BoundaryNorm
import matplotlib.pyplot as plt
%matplotlib inline

plt.rc('font', family='serif')
plt.rc('font', size=12)
plt.rc('facecolor', )

# met mast functions and utilities
sys.path.append('../')
import met_funcs as MET
import vis as vis
import utils as utils

## set up data load and figure save paths

In [2]:
# paths (must mount volume smb://nrel.gov/shared/wind/WindWeb/MetData/135mData/)
towerID = 'M5'
metPathLoHz = '/Users/nhamilto/Documents/Wake_Dynamics/SiteChar/data/{}/txt/'.format(towerID)
figPath = '../../figs/{}'.format(towerID)

# time range
years  = [ int(a) for a in np.arange(2017,2018,1) ] #
months = [ int(a) for a in np.arange(1,12.1,1) ]

inputfiles = [] # list of files to be read into metdata object
filecount = 0

for year in years:
    for month in months:
        fName = glob.glob(os.path.join(metPathLoHz,'{0}_{1}.txt'.format(year,month_name[month])))        
        if len(fName)>0:
            fName = fName[0] 
            inputfiles.append(fName)
print('{} files to be read into MetData'.format(len(inputfiles)))

12 files to be read into MetData


## Load data
### remove columns that are all NaN
### remove columns that are not in the list of variables we care about

In [3]:
metdat = MET.load_met_data(inputfiles, verbose=False)

metdat = MET.drop_nan_cols(metdat)
# simply applies Pandas DataFrame method:
# metdat.dropna(axis=1,how='all', inplace=True)

keepcols = MET.categories_to_keep()
keepcols = [col for col in metdat.columns 
            if col.split(' (')[0].lower() in keepcols 
            if '.1' not in col]
dropcols = [col for col in metdat.columns if col not in keepcols]

metdat.drop(dropcols,axis=1,inplace=True)

## apply qc flags to data

most data channels have a respective quality control column,

``` <data channel A>, <QC channel A>```

That communicates whether the respective 10-minute binned data point should be considered valid (`qc=1`), questionable (`qc=0`), or should be rejected (`qc=-1`).

A QC mask must be applied to each data channel individually, as we don't want to exclude a point from `<data channel A>` based on `<QC channel not(A)>`

In [4]:
metdat = MET.qc_mask(metdat)

number of data columns: 100
number of QC columns: 106


## add a new category 'Stability flag' 
calculated from Monin-Obukhov length at various heights

In [5]:
## flag data by stability class
stabconds, stabcat = MET.flag_stability(metdat)

## caterogize data channels remaining
identify different categories
assign each units, labels, and savenames

In [6]:
## group columns based on category, assign units, labels, savenames
varcats, varunits, varlabels, varsave = MET.categorize_fields(metdat, keeplist=True)

## final, ad-hoc tuning
filter out turbulence for wind speed < 1 m/s

filter obukhov length > 2000

filter air temp to be in degrees C

filter gradient richardson number > 20


In [7]:
## drop columns not in any of the categories, filter TI, temperature, stability parameters
MET.groom_data(metdat, varcats)

## outlier detection on remaining data

In [8]:
## Finally, reject outliers more than 5 standard deviations from the mean
for col in metdat.columns:
    try:
        metdat[col] = MET.reject_outliers(metdat[col], m=6)
    except:
        continue

## combine categorical information into a dictionary for easy use

In [9]:
catinfo = {}
catinfo['columns'] = varcats
catinfo['units'] = varunits
catinfo['labels'] = varlabels
catinfo['save'] = varsave

In [10]:
# M5 excluded angles,
# Obstruction,	start ang.	end ang.
# GE			46		146
# Alstom		123		203
# CART-3		172		213
# CART-2		177		212
# Siemens		165		210
# Gamesa		189		228
exclude_angles = [(46,146),(123,203),(172,213),(177,212),(165,210),(189,228)]

# Shortcut version 
Added a new function in MET called 'fix_data_for_transfer' that includes all of the above QC and filtering.

In [ ]:
metdat = MET.load_met_data(inputfiles, verbose=False)

metdat, catinfo = MET.fix_data_for_transfer(metdat)